In [3]:
import rebound
import reboundx
import numpy as np
sim = rebound.Simulation()
sim.add(m=1., hash="Sun")
sim.add(a=1., e=0.5, f=np.pi)
rebx= reboundx.Extras(sim)
ps = sim.particles

In [4]:
effect = rebx.add("track_min_distance")

In [5]:
ps[1].params["min_distance"] = 5.

In [6]:
sim.integrate(10.)

In [7]:
ps[1].params["min_distance"]

0.5000213256253688